In [4]:
pip install pandas numpy scikit-learn nltk joblib flask


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import joblib
from flask import Flask, request, jsonify

In [5]:
import pandas as pd

# Define file paths
phishing_email_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/phishing_email.csv'
nigerian_fraud_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/Nigerian_Fraud.csv'
spam_assassin_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/SpamAssasin.csv'
ceas_08_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/CEAS_08.csv'
enron_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/Enron.csv'
ling_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/Ling.csv'
nazario_path = 'C:/Users/Samarth Seshadri/Documents/IS Project/datasets/Nazario.csv'

# Load datasets
phishing_email = pd.read_csv(phishing_email_path)
nigerian_fraud = pd.read_csv(nigerian_fraud_path)
spam_assassin = pd.read_csv(spam_assassin_path)
ceas_08 = pd.read_csv(ceas_08_path)
enron = pd.read_csv(enron_path)
ling = pd.read_csv(ling_path)
nazario = pd.read_csv(nazario_path)


In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Download NLTK stop words and WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Initialize an empty DataFrame for the combined data
combined_data = pd.DataFrame(columns=['text', 'label'])

# Combine datasets into a single DataFrame
datasets = [phishing_email, nigerian_fraud, spam_assassin, ceas_08, enron, ling, nazario]

for dataset in datasets:
    if 'text_combined' in dataset.columns:
        dataset_cleaned = dataset.rename(columns={'text_combined': 'text'})[['text', 'label']]
    elif 'body' in dataset.columns:
        dataset_cleaned = dataset.rename(columns={'body': 'text'})[['text', 'label']]
    elif 'content' in dataset.columns:
        dataset_cleaned = dataset.rename(columns={'content': 'text'})[['text', 'label']]
    combined_data = pd.concat([combined_data, dataset_cleaned], ignore_index=True)

# Clean the text data
def clean_text(text):
    text = re.sub('<.*?>', '', text)  # Remove HTML tags
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords.words('english'))
    return text

combined_data['text'] = combined_data['text'].astype(str).fillna('').apply(clean_text)

# Ensure labels are integers
combined_data['label'] = combined_data['label'].astype(int)

# Feature extraction using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(combined_data['text']).toarray()
y = combined_data['label'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Confirm shapes of the training and testing sets
print("Shapes of the datasets:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


[nltk_data] Downloading package stopwords to C:\Users\Samarth
[nltk_data]     Seshadri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samarth
[nltk_data]     Seshadri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Shapes of the datasets:
X_train: (131977, 5000)
X_test: (32995, 5000)
y_train: (131977,)
y_test: (32995,)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Random Forest model
random_forest = RandomForestClassifier()

# Train the model
random_forest.fit(X_train, y_train)

# Make predictions
y_pred = random_forest.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Random Forest Accuracy: 0.9922412486740415
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     15791
           1       0.99      0.99      0.99     17204

    accuracy                           0.99     32995
   macro avg       0.99      0.99      0.99     32995
weighted avg       0.99      0.99      0.99     32995



In [11]:
import joblib

# Save the trained model
joblib.dump(random_forest, 'phishing_detection_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [12]:
# Write the Flask app to a Python script file
flask_code = """
import joblib
from flask import Flask, request, jsonify

# Load the saved model
model = joblib.load('phishing_detection_model.pkl')

# Function to clean and preprocess the input text
def clean_text(text):
    import re
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = re.sub('<.*?>', '', text)  # Remove HTML tags
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords.words('english'))
    return text

# Load the TF-IDF vectorizer
tfidf = joblib.load('tfidf_vectorizer.pkl')

# Create Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    email_text = request.json['text']
    email_text = clean_text(email_text)
    features = tfidf.transform([email_text]).toarray()
    prediction = model.predict(features)
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)
"""

# Save the Flask app code to a Python file
with open('app.py', 'w') as f:
    f.write(flask_code)


In [13]:
import requests
import json

url = 'http://127.0.0.1:5000/predict'
payload = {
    'text': "Congratulations! You have won a $1000 gift card. Click here to claim your prize."
}
headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(payload), headers=headers)

print(f'Status Code: {response.status_code}')
print(f'Response: {response.json()}')


Status Code: 200
Response: {'prediction': 1}


In [14]:
payload = {
    'text': "Your account has been compromised. Click here to reset your password."
}
response = requests.post(url, data=json.dumps(payload), headers=headers)
print(f'Status Code: {response.status_code}')
print(f'Response: {response.json()}')


Status Code: 200
Response: {'prediction': 1}


In [16]:
import joblib
from flask import Flask, request, jsonify
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load the saved model
model = joblib.load('phishing_detection_model.pkl')

# Load the TF-IDF vectorizer
tfidf = joblib.load('tfidf_vectorizer.pkl')

# Create Flask app
app = Flask(__name__)

# Function to clean and preprocess the input text
def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    text = re.sub('<.*?>', '', text)  # Remove HTML tags
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stopwords.words('english'))
    return text

@app.route('/predict', methods=['POST'])
def predict():
    if not request.json or 'text' not in request.json:
        return jsonify({'error': 'Invalid input'}), 400

    email_text = request.json['text']
    email_text = clean_text(email_text)
    features = tfidf.transform([email_text]).toarray()
    prediction = model.predict(features)
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  # use_reloader=False to prevent Jupyter from restarting the kernel


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
